In [1]:
import sys
sys.path.append('../')
from python_files import main

import pandas as pd
import plotly.express as px

In [2]:
confirmed_global, deaths_global, recovered_global, country_cases = main.collect_data()

In [3]:
def unpivot(df):
   return df.melt(id_vars = ["country"],  value_vars = df.columns[1:])


In [4]:
def take_top10(df):
  top = list(df[df["variable"] == df['variable'][df.index[-1]]].sort_values(by=['value'], ascending=False).head(10)["country"])
  df = df[df['country'].isin(top)]
  return df

In [5]:
def create_data(df):
  new = take_top10(df)
  l = list(set(new["variable"]))
  l.sort()
  l.reverse()
  ff= new[new['variable'].isin(l[::5])]
  ff.rename(columns = {"country": "Country","variable" : "Date","value": "Cases"}, inplace = True)
  return ff

In [12]:
def plot_fig(ff):
  fig = px.bar(ff, x="Country", y="Cases", color="Country",template="plotly_dark",animation_frame="Date", animation_group="Country", range_y=[0,ff["Cases"].max()])
  fig.layout.update(showlegend=False)
  return fig


In [13]:
def animated_barchart(df):
  return (plot_fig(create_data(take_top10(unpivot(df)))))

In [14]:
animated_barchart(confirmed_global)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [54]:
def compare(df,*args):
  l = list(args)
  temp = unpivot(df)
  temp = temp[temp["country"].isin(l)]
  return temp


In [82]:
def plot_fig_compare(ff):
  fig = px.bar(ff, x="Country", y="Cases", color="Country",template="plotly_dark",animation_frame="Date", animation_group="Country",range_y=[0,ff["Cases"].max()])
  fig.layout.update(hovermode = "x")
  return fig

In [83]:
def create_comparison(df,*args):
  df = compare(df,*args)
  ff = create_data(df)
  return plot_fig_compare(ff)

In [84]:
create_comparison(confirmed_global,"India","US","Australia")

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [89]:
def static_line(df,*args):
  df = compare(df,*args)
  ff = create_data(df)
  fig = px.line(ff, x="Date", y="Cases", color="Country",template="plotly_dark",range_y=[0,ff["Cases"].max()])
  fig.layout.update(hovermode = "x")
  return fig


In [91]:
static_line(recovered_global,"India","New Zealand","US")

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
def line_comparison(country):
    whole_df = pd.DataFrame()
    whole_df["dates"] = list(confirmed_global.columns[1:])
    whole_df["confirmed"] = list(confirmed_global.loc[confirmed_global['country'] == country].values.flatten()[1:])
    whole_df["deaths"] = list(deaths_global.loc[deaths_global['country'] == country].values.flatten()[1:])
    whole_df["recovered"] = list(recovered_global.loc[recovered_global['country'] == country].values.flatten()[1:]) 
    
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=whole_df["dates"],
            y=whole_df["confirmed"],
            mode="lines",
            name="confirmed"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=whole_df["dates"],
            y=whole_df["deaths"],
            mode="lines",
            name="deaths"
        )
    )

    fig.add_trace(
        go.Scatter(
            x=whole_df["dates"],
            y=whole_df["recovered"],
            mode="lines",
            name="recovered"
        )
    )


    fig.update_layout(
        height=500,
        showlegend=True,
        xaxis = { 'showgrid': False }, yaxis = { 'showgrid': False },
        template = "plotly_dark",
        title_text=f"Analysis of {country.title()}", hovermode='x'
    )

    return fig

In [ ]:
line_comparison("India")